In [72]:
from datetime import datetime, timedelta, date
from sqlalchemy import create_engine
import numpy as np
import calendar
from datetime import date, timedelta, datetime
from pymongo import MongoClient
import json
from dateutil.relativedelta import relativedelta
import sys
import re
import pandas as pd
import psycopg2
from os.path import expanduser
from google.cloud import bigquery
from google.oauth2 import service_account
from bson.objectid import ObjectId
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')
# time zones
from dateutil import tz
from_zone = tz.gettz('UTC')
to_zone = tz.gettz('Asia/Kolkata')



In [73]:
from pandas import json_normalize
import requests

today = date.today().strftime('%Y-%m-%d')
 
response = requests.get(
  'https://api.therainery.com/astronomy/sun',
  params={
    'latitude': 18.51957,
    'longitude': 73.85535,
    'start': today,
    'end': today
  },
  headers={
    'x-api-key': 'aYpwCCeYdw1xxdeUzLqFu7c8uIkVO7pya5bCOwOr'
  }
)
# response.text
df_therainery = json_normalize(response.json(),'data')
df_therainery

,date,astronomicalDawn,astronomicalDusk,civilDawn,civilDusk,goldenHourEveningStart,goldenHourMorningEnd,nadir,nauticalDawn,nauticalDusk,solarNoon,sunriseStart,sunriseEnd,sunsetStart,sunsetEnd
0,2021-05-21,2021-05-20T23:10:24Z,2021-05-21T14:54:48Z,2021-05-21T00:06:56Z,2021-05-21T13:58:16Z,2021-05-21T13:03:53Z,2021-05-21T01:01:18Z,2021-05-20T19:02:36Z,2021-05-20T23:39:01Z,2021-05-21T14:26:11Z,2021-05-21T07:02:36Z,2021-05-21T00:30:33Z,2021-05-21T00:32:58Z,2021-05-21T13:32:14Z,2021-05-21T13:34:39Z


In [74]:
df_therainery = df_therainery[['date','sunriseStart','sunriseEnd','sunsetStart','sunsetEnd']]
df_therainery

,date,sunriseStart,sunriseEnd,sunsetStart,sunsetEnd
0,2021-05-21,2021-05-21T00:30:33Z,2021-05-21T00:32:58Z,2021-05-21T13:32:14Z,2021-05-21T13:34:39Z


In [75]:
df_therainery['sunriseStart_ist'] = pd.to_datetime(df_therainery['sunriseStart']) + pd.to_timedelta(19800, unit='s')
df_therainery['sunriseEnd_ist'] = pd.to_datetime(df_therainery['sunriseEnd']) + pd.to_timedelta(19800, unit='s')
df_therainery['sunsetStart_ist'] = pd.to_datetime(df_therainery['sunsetStart']) + pd.to_timedelta(19800, unit='s')
df_therainery['sunsetEnd_ist'] = pd.to_datetime(df_therainery['sunsetEnd']) + + pd.to_timedelta(19800, unit='s')

df_therainery['sunriseStart_ist'] = df_therainery['sunriseStart_ist'].dt.strftime("%Y-%m-%d %H:%M:%S")
df_therainery['sunriseEnd_ist'] = df_therainery['sunriseEnd_ist'].dt.strftime("%Y-%m-%d %H:%M:%S")
df_therainery['sunsetStart_ist'] = df_therainery['sunsetStart_ist'].dt.strftime("%Y-%m-%d %H:%M:%S")
df_therainery['sunsetEnd_ist'] = df_therainery['sunsetEnd_ist'].dt.strftime("%Y-%m-%d %H:%M:%S")

df_therainery['city'] = 'Pune'
df_therainery['latitude'] = '18.51957'
df_therainery['longitude'] = '73.85535'

df_therainery = df_therainery[['date','city','latitude','longitude','sunriseStart_ist','sunriseEnd_ist','sunsetStart_ist','sunsetEnd_ist']]
df_therainery

,date,city,latitude,longitude,sunriseStart_ist,sunriseEnd_ist,sunsetStart_ist,sunsetEnd_ist
0,2021-05-21,Pune,18.51957,73.85535,2021-05-21 06:00:33,2021-05-21 06:02:58,2021-05-21 19:02:14,2021-05-21 19:04:39


In [76]:
# load_historical_data():
response = requests.get(
  'https://api.therainery.com/astronomy/sun',
  params={
    'latitude': 18.51957,
    'longitude': 73.85535,
    'start': '2020-01-01',
    'end': '2021-05-19'
  },
  headers={
    'x-api-key': 'aYpwCCeYdw1xxdeUzLqFu7c8uIkVO7pya5bCOwOr'
  }
)
df_load_load_historical_data =  json_normalize(response.json(),'data')



In [77]:
df_load_load_historical_data = df_load_load_historical_data[['date','sunriseStart','sunriseEnd','sunsetStart','sunsetEnd']]


In [78]:
df_load_load_historical_data['sunriseStart_ist'] = pd.to_datetime(df_load_load_historical_data['sunriseStart']) + pd.to_timedelta(19800, unit='s')
df_load_load_historical_data['sunriseEnd_ist'] = pd.to_datetime(df_load_load_historical_data['sunriseEnd']) + pd.to_timedelta(19800, unit='s')
df_load_load_historical_data['sunsetStart_ist'] = pd.to_datetime(df_load_load_historical_data['sunsetStart']) + pd.to_timedelta(19800, unit='s')
df_load_load_historical_data['sunsetEnd_ist'] = pd.to_datetime(df_load_load_historical_data['sunsetEnd']) + + pd.to_timedelta(19800, unit='s')

df_load_load_historical_data['sunriseStart_ist'] = df_load_load_historical_data['sunriseStart_ist'].dt.strftime("%Y-%m-%d %H:%M:%S")
df_load_load_historical_data['sunriseEnd_ist'] = df_load_load_historical_data['sunriseEnd_ist'].dt.strftime("%Y-%m-%d %H:%M:%S")
df_load_load_historical_data['sunsetStart_ist'] = df_load_load_historical_data['sunsetStart_ist'].dt.strftime("%Y-%m-%d %H:%M:%S")
df_load_load_historical_data['sunsetEnd_ist'] = df_load_load_historical_data['sunsetEnd_ist'].dt.strftime("%Y-%m-%d %H:%M:%S")
df_load_load_historical_data['city'] = 'Pune'
df_load_load_historical_data['latitude'] = '18.51957'
df_load_load_historical_data['longitude'] = '73.85535'


In [79]:

df_load_load_historical_data = df_load_load_historical_data[['date','city','latitude','longitude','sunriseStart_ist','sunriseEnd_ist','sunsetStart_ist','sunsetEnd_ist']]
df_load_load_historical_data

,date,city,latitude,longitude,sunriseStart_ist,sunriseEnd_ist,sunsetStart_ist,sunsetEnd_ist
0,2020-01-01,Pune,18.51957,73.85535,2020-01-01 07:07:49,2020-01-01 07:10:17,2020-01-01 18:07:28,2020-01-01 18:09:56
1,2020-01-02,Pune,18.51957,73.85535,2020-01-02 07:08:10,2020-01-02 07:10:37,2020-01-02 18:08:03,2020-01-02 18:10:31
2,2020-01-03,Pune,18.51957,73.85535,2020-01-03 07:08:29,2020-01-03 07:10:56,2020-01-03 18:08:39,2020-01-03 18:11:07
3,2020-01-04,Pune,18.51957,73.85535,2020-01-04 07:08:47,2020-01-04 07:11:14,2020-01-04 18:09:15,2020-01-04 18:11:43
4,2020-01-05,Pune,18.51957,73.85535,2020-01-05 07:09:04,2020-01-05 07:11:31,2020-01-05 18:09:52,2020-01-05 18:12:19
...,...,...,...,...,...,...,...,...
96,2020-04-06,Pune,18.51957,73.85535,2020-04-06 06:25:44,2020-04-06 06:28:00,2020-04-06 18:48:07,2020-04-06 18:50:23
97,2020-04-07,Pune,18.51957,73.85535,2020-04-07 06:24:55,2020-04-07 06:27:11,2020-04-07 18:48:20,2020-04-07 18:50:36
98,2020-04-08,Pune,18.51957,73.85535,2020-04-08 06:24:08,2020-04-08 06:26:24,2020-04-08 18:48:34,2020-04-08 18:50:50
99,2020-04-09,Pune,18.51957,73.85535,2020-04-09 06:23:20,2020-04-09 06:25:37,2020-04-09 18:48:48,2020-04-09 18:51:04


In [80]:
# # isolate the new indexes, (present in live_df but not in hist_df)
# new_locs = ~live_df.index.isin(hist_df.index)

# # append just the new entries in the live_df
# new_df = hist_df.append(live_df.loc[new_locs])

In [81]:
# load todays data
def load_today_data():
    today = date.today().strftime('%Y-%m-%d')
 
    response = requests.get(
      'https://api.therainery.com/astronomy/sun',
      params={
        'latitude': 18.51957,
        'longitude': 73.85535,
        'start': today,
        'end': today
      },
      headers={
        'x-api-key': 'aYpwCCeYdw1xxdeUzLqFu7c8uIkVO7pya5bCOwOr'
      }
    )
    # response.text
    df_therainery = json_normalize(response.json(),'data')
    df_therainery = df_therainery[['date','sunriseStart','sunriseEnd','sunsetStart','sunsetEnd']]
    df_therainery['sunriseStart_ist'] = pd.to_datetime(df_therainery['sunriseStart']) + pd.to_timedelta(19800, unit='s')
    df_therainery['sunriseEnd_ist'] = pd.to_datetime(df_therainery['sunriseEnd']) + pd.to_timedelta(19800, unit='s')
    df_therainery['sunsetStart_ist'] = pd.to_datetime(df_therainery['sunsetStart']) + pd.to_timedelta(19800, unit='s')
    df_therainery['sunsetEnd_ist'] = pd.to_datetime(df_therainery['sunsetEnd']) + + pd.to_timedelta(19800, unit='s')

    df_therainery['sunriseStart_ist'] = df_therainery['sunriseStart_ist'].dt.strftime("%Y-%m-%d %H:%M:%S")
    df_therainery['sunriseEnd_ist'] = df_therainery['sunriseEnd_ist'].dt.strftime("%Y-%m-%d %H:%M:%S")
    df_therainery['sunsetStart_ist'] = df_therainery['sunsetStart_ist'].dt.strftime("%Y-%m-%d %H:%M:%S")
    df_therainery['sunsetEnd_ist'] = df_therainery['sunsetEnd_ist'].dt.strftime("%Y-%m-%d %H:%M:%S")
    
    df_therainery['city'] = 'Pune'
    df_therainery['latitude'] = '18.51957'
    df_therainery['longitude'] = '73.85535'

    df_therainery = df_therainery[['date','city','latitude','longitude','sunriseStart_ist','sunriseEnd_ist','sunsetStart_ist','sunsetEnd_ist']]
    return df_therainery

In [82]:
df_append_today = load_today_data()
df_final = df_load_load_historical_data.append(df_append_today)
df_final

,date,city,latitude,longitude,sunriseStart_ist,sunriseEnd_ist,sunsetStart_ist,sunsetEnd_ist
0,2020-01-01,Pune,18.51957,73.85535,2020-01-01 07:07:49,2020-01-01 07:10:17,2020-01-01 18:07:28,2020-01-01 18:09:56
1,2020-01-02,Pune,18.51957,73.85535,2020-01-02 07:08:10,2020-01-02 07:10:37,2020-01-02 18:08:03,2020-01-02 18:10:31
2,2020-01-03,Pune,18.51957,73.85535,2020-01-03 07:08:29,2020-01-03 07:10:56,2020-01-03 18:08:39,2020-01-03 18:11:07
3,2020-01-04,Pune,18.51957,73.85535,2020-01-04 07:08:47,2020-01-04 07:11:14,2020-01-04 18:09:15,2020-01-04 18:11:43
4,2020-01-05,Pune,18.51957,73.85535,2020-01-05 07:09:04,2020-01-05 07:11:31,2020-01-05 18:09:52,2020-01-05 18:12:19
...,...,...,...,...,...,...,...,...
97,2020-04-07,Pune,18.51957,73.85535,2020-04-07 06:24:55,2020-04-07 06:27:11,2020-04-07 18:48:20,2020-04-07 18:50:36
98,2020-04-08,Pune,18.51957,73.85535,2020-04-08 06:24:08,2020-04-08 06:26:24,2020-04-08 18:48:34,2020-04-08 18:50:50
99,2020-04-09,Pune,18.51957,73.85535,2020-04-09 06:23:20,2020-04-09 06:25:37,2020-04-09 18:48:48,2020-04-09 18:51:04
100,2020-04-10,Pune,18.51957,73.85535,2020-04-10 06:22:33,2020-04-10 06:24:50,2020-04-10 18:49:02,2020-04-10 18:51:19


In [83]:
# engine = create_engine('sqlite:///test.db', echo=True)
# sqlite_connection = engine.connect()

In [84]:
# sqlite_table = "the-rainery-pune"
# df_final.to_sql(sqlite_table, sqlite_connection, if_exists='append', index=False)

sql_engine = create_engine('sqlite:///test.db', echo=False)
connection = sql_engine.raw_connection()
df_final.to_sql("the-rainery-pune", connection,index=False, if_exists='append')

In [85]:
# df_final.to_csv('test_db.csv', header=False, index=False)